**Train Data**

In [17]:
#Training data

import csv

# Initialize an empty dictionary
train_data_dict = {}
train_data_dict["Sentence"]=[]
train_data_dict["Toxic_spans"]=[]
train_data_dict["Label"]=[]

# Open the CSV file
with open('tsd_train.csv', mode='r') as file:
    # Create a CSV reader object
    reader = csv.reader(file)
    reader=list(reader)
    # Iterate over each row in the CSV file
    for i in range(1,len(reader)):
        row=reader[i]
        #print(row)
        train_data_dict["Sentence"].append(row[1])
        s=row[0][1:]
        s=s[:-1]
        span_list=s.split(',')
        span_list=[''.join(j.split()) for j in span_list]
        #print(span_list)
        span_list=[j for j in span_list if j!='']
        span_list=[int(j) for j in span_list]
        train_data_dict["Toxic_spans"].append(span_list)
        if row[0]!="[]":
            train_data_dict["Label"].append('toxic')
        else:
            train_data_dict["Label"].append("not toxic")



In [7]:
len(train_data_dict["Label"])

7939

In [ ]:
train_data_dict["Toxic_spans"]

In [9]:
train_data_dict["Label"].count('toxic')

7454

BIO Encoding

In [59]:
def BIO_encoding(input_data):
    Encoded_dict={}     #Dictionary to store BIO encoding

    for i in range(len(input_data["Sentence"])):
        d={}
        d["Text"]=input_data["Sentence"][i]
        Bio_labels=[]   #store BIO code
        if input_data["Label"][i]=='toxic':
            indices=input_data["Toxic_spans"][i]

            #Currently assuming the toxic span is continuous(CHECK THIS AND CHANGE IF NEEDED)
            #print(d["Text"])
            #print(indices)
            prev=d["Text"][:indices[0]].split()     #part before toxic span
            Bio_labels+=['O']*len(prev)     #O label for non-toxic part

            #now for toxic part
            k=len(indices)
            start=0
            end=1
            prev_ind=indices[start]
            while end<k:
                if indices[end]>prev_ind+1:
                    #print(start,end,prev_ind)
                    span=d["Text"][indices[start]:(indices[end-1]+1)]    #extract that span of text
                    tokens=span.split()     #break them into tokens/words
                    next=d["Text"][indices[end-1]+1:indices[end]].split()  #part after span
                    Bio_labels.append('B')          #B label to start span
                    Bio_labels+=['I']*(len(tokens)-1)   #I labels for rest of the span
                    Bio_labels+=['O']*len(next)     #O labels for remaining non-toxic part
                    start=end
                    end=start+1
                    prev_ind=indices[start]
                else:
                    end+=1
                    prev_ind+=1
            #print(indices)
            #print(start,end)
            span=d["Text"][indices[start]:(indices[end-1]+1)]    #extract that span of text
            tokens=span.split()     #break them into tokens/words

            Bio_labels.append('B')          #B label to start span
            Bio_labels+=['I']*(len(tokens)-1)   #I labels for rest of the span
            next=d["Text"][indices[end-1]+1:].split()  #part after span
            Bio_labels+=['O']*len(next)     #O labels for remaining non-toxic part

        else:
            Bio_labels=['O']*len(d["Text"].split())

        d["Bio_labels"]=Bio_labels  #add to dict
        Encoded_dict[i+1]=d

    return Encoded_dict


In [60]:
BIO_codes_train=BIO_encoding(train_data_dict)

In [61]:
BIO_codes_train[1]

{'Text': 'Another violent and aggressive immigrant killing a innocent and intelligent US Citizen.... Sarcasm',
 'Bio_labels': ['O',
  'B',
  'I',
  'I',
  'I',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O']}

In [67]:
BIO_codes_train[9]

{'Text': "Obamacare is on it's last gasping breaths.   You idiots who don't want something else passed that's going to save your free healthcare are kind of stupid.   But if you block anything else that gives you socialized healthcare it's your own fault for being unbudging.  You can go back to nothing when nothing further is passed and Obamacare is just an ugly memory.",
 'Bio_labels': ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O']}

In [71]:
#Save encoded dictionary

import json
train_set=json.dumps(BIO_codes_train,indent=4)

with open('BIO_train.json','w') as f:
    f.write(train_set)